In [ ]:
#note that training was done on the following notebook using different hyperparameters and different efficientnet bases
#https://www.kaggle.com/danielwijaya/cassava-v1?scriptVersionId=50195139

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from functools import partial
from sklearn.model_selection import train_test_split
import re
import random
import sys
sys.path.append('/kaggle/input/efficient-net/')
sys.path.append('/kaggle/input/kerasapplications/')
import keras_applications
import efficientnet.keras as efn

In [ ]:
#load pre-trained models
model_15 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB6_based(15).h5')
model_17 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB4_based(17).h5')
##model_18 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(18).h5')
##model_19 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(19).h5')
##model_20 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(20).h5')
model_21 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(21).h5')
##model_24 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB4_based(24).h5')
##model_25 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(25).h5')
#model_26 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(26).h5')
model_28_1 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(28-1).h5')
#model_28_2 = tf.keras.models.load_model('../input/cassava-trained/EfficientNetB5_based(28-2).h5')


In [ ]:
test_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
print(test_df)
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = '../input/cassava-leaf-disease-classification'#KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
BATCH_SIZE = 16*8#strategy.num_replicas_in_sync 
IMAGE_SIZE = [512,512]
CLASSES = ["1", "2", "3", "4", "5"]
def dataset_sizes(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return(np.sum(n))
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH+"/test_tfrecords/ld_test*.tfrec")
NUM_TEST_IMAGES = dataset_sizes(TEST_FILENAMES)

In [ ]:
def to_float32(image, label):
    return tf.cast(image, tf.float32), label

#load data

# decode data 
def decode_img(img):
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.cast(img, tf.float32)/255.0
    img = tf.reshape(img, [*IMAGE_SIZE, 3])
    return img
# read tfrecords
def read_tfrecord(example, labeled):
    if labeled:
        TFREC_FORMAT = {
            "image": tf.io.FixedLenFeature([], tf.string), #[] is the shape (is just single value)
            "target": tf.io.FixedLenFeature([], tf.int64)
        }
    else:
        TFREC_FORMAT = {
            "image": tf.io.FixedLenFeature([], tf.string),
            "image_name": tf.io.FixedLenFeature([], tf.string)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    img = decode_img(example["image"])
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        return(img, label)
    else:
        idNum = example["image_name"]
        return(img, idNum)
# load dataset
def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False #disable order == increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTOTUNE) #automatically interleaves reads from multiple files
    dataset.with_options(ignore_order) #use data as it streams in
    dataset = dataset.map(partial(read_tfrecord, labeled = labeled), num_parallel_calls = AUTOTUNE)
    return dataset
def get_test_data(ordered = False): #for outputting actual test data, will need to specify ordered = True
    dataset = load_dataset(filenames = TEST_FILENAMES, labeled = False, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
test_ds = get_test_data(ordered=True) 
test_ds = test_ds.map(to_float32)

testing_dataset = get_test_data()
testing_dataset = testing_dataset.unbatch().batch(1)

print('Computing predictions...')
test_images_ds = testing_dataset
test_images_ds = test_ds.map(lambda image, idnum: image)
prob1 = model_15.predict(test_images_ds)
prob2 = model_17.predict(test_images_ds)
##prob3 = model_18.predict(test_images_ds)
#prob4 = model_19.predict(test_images_ds)
#prob5 = model_20.predict(test_images_ds)
prob6 = model_21.predict(test_images_ds)
##prob7 = model_24.predict(test_images_ds)
##prob8 = model_25.predict(test_images_ds)
#prob9 = model_26.predict(test_images_ds)
prob10 = model_28_1.predict(test_images_ds)
#prob11 = model_28_2.predict(test_images_ds)
probabilities = 0.25*(prob1+prob2+prob6+prob10)
predictions = np.argmax(probabilities, axis = -1)
print(predictions)

In [ ]:
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='image_id,label', comments='')